In [62]:
import sys
sys.path.append('../data/cifar100/')
import torch
import torch.nn as nn
import torch.optim as optim
from copy import deepcopy
import random
from torch.utils.data import Subset
from statistics import mean
#from cifar100_loader import load_cifar100
#from models.model import LeNet5 #import the model
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
from collections import Counter
import warnings
import string
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import itertools
from copy import deepcopy
import collections
from sklearn.model_selection import train_test_split
import json
import random
import re
import torch.optim as optim
from collections import defaultdict
import kagglehub

### Constants for FL training

In [63]:
# Constants for FL training
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

NUM_CLIENTS = 100  # Total number of clients in the federation
FRACTION_CLIENTS = 0.1  # Fraction of clients selected per round (C)
LOCAL_EPOCHS = 4  # Number of local steps (J)
GLOBAL_ROUNDS = 100  # Total number of communication rounds

BATCH_SIZE = 64  # Batch size for local training
LR = 1e-3  # Initial learning rate for local optimizers
MOMENTUM = 0.9  # Momentum for SGD optimizer
WEIGHT_DECAY = 5e-5  # Regularization term for local training

LOG_FREQUENCY = 10  # Frequency of logging training progress

# Loaders

In [64]:
CHARACTER_RE = re.compile(r'^  ([a-zA-Z][a-zA-Z ]*)\. (.*)')  # Matches character lines
CONT_RE = re.compile(r'^    (.*)')  # Matches continuation lines
COE_CHARACTER_RE = re.compile(r'^([a-zA-Z][a-zA-Z ]*)\. (.*)')  # Special regex for Comedy of Errors
COE_CONT_RE = re.compile(r'^(.*)')  # Continuation for Comedy of Errors

def parse_shakespeare_file(filepath):
    """
    Reads and splits Shakespeare's text into plays, characters, and their dialogues.
    Returns training and test datasets based on the specified fraction.
    """
    with open(filepath, "r") as f:
        content = f.read()
    plays, _ = _split_into_plays(content)  # Split the text into plays
    _, train_examples, test_examples = _get_train_test_by_character(
        plays, test_fraction=1 - TRAIN_FRACTION
    )
    return train_examples, test_examples

def _split_into_plays(shakespeare_full):
    """
    Splits the full Shakespeare text into individual plays and characters' dialogues.
    Handles special parsing for "The Comedy of Errors".
    """
    plays = []
    slines = shakespeare_full.splitlines(True)[1:]  # Skip the first line (title/header)
    current_character = None
    comedy_of_errors = False

    for i, line in enumerate(slines):
        # Detect play titles and initialize character dictionary
        if "by William Shakespeare" in line:
            current_character = None
            characters = defaultdict(list)
            title = slines[i - 2].strip() if slines[i - 2].strip() else slines[i - 3].strip()
            comedy_of_errors = title == "THE COMEDY OF ERRORS"
            plays.append((title, characters))
            continue

        # Match character lines or continuation lines
        match = _match_character_regex(line, comedy_of_errors)
        if match:
            character, snippet = match.group(1).upper(), match.group(2)
            if not (comedy_of_errors and character.startswith("ACT ")):
                characters[character].append(snippet)
                current_character = character
        elif current_character:
            match = _match_continuation_regex(line, comedy_of_errors)
            if match:
                characters[current_character].append(match.group(1))

    # Filter out plays with insufficient dialogue data
    return [play for play in plays if len(play[1]) > 1], []

def _match_character_regex(line, comedy_of_errors=False):
    """Matches character dialogues, with special handling for 'The Comedy of Errors'."""
    return COE_CHARACTER_RE.match(line) if comedy_of_errors else CHARACTER_RE.match(line)

def _match_continuation_regex(line, comedy_of_errors=False):
    """Matches continuation lines of dialogues."""
    return COE_CONT_RE.match(line) if comedy_of_errors else CONT_RE.match(line)

def _get_train_test_by_character(plays, test_fraction=0.2):
    """
    Splits dialogues by characters into training and testing datasets.
    Ensures each character has at least one example in the training set.
    """
    all_train_examples = defaultdict(list)
    all_test_examples = defaultdict(list)

    def add_examples(example_dict, example_tuple_list):
        """Adds examples to the respective dataset dictionary."""
        for play, character, sound_bite in example_tuple_list:
            example_dict[f"{play}_{character}".replace(" ", "_")].append(sound_bite)

    for play, characters in plays:
        for character, sound_bites in characters.items():
            examples = [(play, character, sound_bite) for sound_bite in sound_bites]
            if len(examples) <= 2:
                continue

            # Calculate the number of test samples
            num_test = max(1, int(len(examples) * test_fraction))
            num_test = min(num_test, len(examples) - 1)  # Ensure at least one training example

            # Split into train and test sets
            train_examples = examples[:-num_test]
            test_examples = examples[-num_test:]

            add_examples(all_train_examples, train_examples)
            add_examples(all_test_examples, test_examples)

    return {}, all_train_examples, all_test_examples

def letter_to_vec(c, n_vocab=128):
    """Converts a single character to a vector index based on the vocabulary size."""
    return ord(c) % n_vocab

def word_to_indices(word, n_vocab=128):
    """
    Converts a word or list of words into a list of indices.
    Each character is mapped to an index based on the vocabulary size.
    """
    if isinstance(word, list):  # If input is a list of words
        res = []
        for stringa in word:
            res.extend([ord(c) % n_vocab for c in stringa])  # Convert each word to indices
        return res
    else:  # If input is a single word
        return [ord(c) % n_vocab for c in word]

def process_x(raw_x_batch, seq_len, n_vocab):
    """
    Processes raw input data into padded sequences of indices.
    Ensures all sequences are of uniform length.
    """
    x_batch = [word_to_indices(word, n_vocab) for word in raw_x_batch]
    x_batch = [x[:seq_len] + [0] * (seq_len - len(x)) for x in x_batch]
    return torch.tensor(x_batch, dtype=torch.long)

def process_y(raw_y_batch, seq_len, n_vocab):
    """
    Processes raw target data into padded sequences of indices.
    Similar to process_x but for target labels.
    """
    y_batch = [word_to_indices(word, n_vocab) for word in raw_y_batch]
    y_batch = [y[:seq_len] + [0] * (seq_len - len(y)) for y in y_batch]
    return torch.tensor(y_batch, dtype=torch.long)

def create_batches(data, batch_size, seq_len, n_vocab):
    """
    Creates batches of input and target data from dialogues.
    Each batch contains sequences of uniform length.
    """
    x_batches = []
    y_batches = []
    dialogues = list(data.values())
    random.shuffle(dialogues)  # Shuffle to ensure randomness in batches

    batch = []
    for dialogue in dialogues:
        batch.append(dialogue)
        if len(batch) == batch_size:
            x_batch = process_x(batch, seq_len, n_vocab)
            y_batch = process_y(batch, seq_len, n_vocab)
            x_batches.append(x_batch)
            y_batches.append(y_batch)
            batch = []

    # Add the last batch if it's not full
    if batch:
        x_batch = process_x(batch, seq_len, n_vocab)
        y_batch = process_y(batch, seq_len, n_vocab)
        x_batches.append(x_batch)
        y_batches.append(y_batch)

    return x_batches, y_batches

from torch.utils.data import Dataset, DataLoader



class ShakespeareDataset(Dataset):
    """
    Custom PyTorch Dataset for processing Shakespeare dialogues.
    Converts input data into sequences of indices for input and target processing.
    """
    def __init__(self, data, seq_len, n_vocab):
        """
        Initializes the ShakespeareDataset instance.

        Args:
            data: Dictionary containing dialogues (e.g., train_data or test_data).
            seq_len: Length of sequences to generate for the model.
            n_vocab: Size of the vocabulary for mapping characters to indices.
        """
        self.data = list(data.values())  # Convert the dictionary values to a list
        self.seq_len = seq_len  # Sequence length for the model
        self.n_vocab = n_vocab  # Vocabulary size

    def __len__(self):
        """
        Returns the total number of samples in the dataset.

        Returns:
            int: Number of dialogues in the dataset.
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        Retrieves a single sample (input and target) from the dataset.

        Args:
            idx: Index of the sample to retrieve.

        Returns:
            Tuple: Processed input (x) and target (y) tensors for the model.
        """
        dialogue = self.data[idx]  # Get the dialogue at the specified index
        x = process_x([dialogue], self.seq_len, self.n_vocab)[0]  # Prepare input tensor
        y = process_y([dialogue], self.seq_len, self.n_vocab)[0]  # Prepare target tensor
        return x, y


### Load the dataset

In [65]:
# Download latest version of the shakespeare dataset and save the path
TRAIN_FRACTION = 0.8  # percentage of the training data
N_VOCAB = 128  # Numero di caratteri nel vocabolario (ASCII)
SEQ_LEN = 128  # length of the sequence for the model
path = kagglehub.dataset_download("kewagbln/shakespeareonline")
print("Path to dataset files:", path)
DATA_PATH = os.path.join(path, "t8.shakespeare.txt")
OUTPUT_DIR = "processed_data/"

train_data, test_data = parse_shakespeare_file(DATA_PATH)

train_dataset = ShakespeareDataset(train_data, seq_len=SEQ_LEN, n_vocab=N_VOCAB)
val_dataset = ShakespeareDataset(test_data, seq_len=SEQ_LEN, n_vocab=N_VOCAB)

    # Creazione dei DataLoader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

Path to dataset files: /root/.cache/kagglehub/datasets/kewagbln/shakespeareonline/versions/1


# Models

In [66]:
import torch.nn.functional as F

class CharLSTM(nn.Module):
    """
    Character-level LSTM model for text processing tasks.
    Includes embedding, LSTM, and a fully connected output layer.
    """
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, output_size):
        super(CharLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        """
        Forward pass through the model.
        Outputs log probabilities for each character.
        """
        embedded = self.embedding(x)
        output, hidden = self.lstm(embedded, hidden)
        output = self.fc(output)  # Fully connected layer for output
        output = F.log_softmax(output, dim=-1)  # Log probabilities
        return output, hidden

    def init_hidden(self, batch_size):
        """Initializes hidden and cell states for the LSTM."""
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size),
                torch.zeros(self.num_layers, batch_size, self.hidden_size))

# Training

### Data Sharding for IID (Indipendent and Identically Distributed) FL Simulation

In [67]:
import numpy as np

def sharding(dataset, number_of_clients, number_of_classes=100):
    """
    Function that performs the sharding of the dataset given as input.
    dataset: dataset to be split;
    number_of_clients: the number of partitions we want to obtain;
    number_of_classes: (int) the number of classes inside each partition, or 100 for IID;
    """

    # Validation of input parameters
    if not (1 <= number_of_classes <= 100):
        raise ValueError("number_of_classes should be an integer between 1 and 100")

    # Shuffle dataset indices for randomness
    indices = np.random.permutation(len(dataset))

    # Compute basic partition sizes
    basic_partition_size = len(dataset) // number_of_clients
    remainder = len(dataset) % number_of_clients

    shards = []
    start_idx = 0

    if number_of_classes == 100:  # IID Case
        # Equally distribute indices among clients: we can just randomly assign to each client an equal amount of records
        for i in range(number_of_clients):
            end_idx = start_idx + basic_partition_size + (1 if i < remainder else 0)
            shards.append(Subset(dataset, indices[start_idx:end_idx]))
            start_idx = end_idx
    else:  # non-IID Case
        # Count of each class in the dataset
        from collections import Counter
        target_counts = Counter(target for _, target in dataset)

        # Calculate per client class allocation
        class_per_client = np.random.choice(list(target_counts.keys()), size=number_of_classes, replace=False)
        class_idx = {class_: np.where([target == class_ for _, target in dataset])[0] for class_ in class_per_client}

        # Assign class indices evenly to clients
        for i in range(number_of_clients):
            client_indices = np.array([], dtype=int)
            for class_ in class_per_client:
                n_samples = len(class_idx[class_]) // number_of_clients + (1 if i < remainder else 0)
                client_indices = np.concatenate((client_indices, class_idx[class_][:n_samples]))
                class_idx[class_] = np.delete(class_idx[class_], np.arange(n_samples))

            shards.append(Subset(dataset, indices=client_indices))

    return shards

### Client Local Update

In [68]:
import time
from torch.backends import cudnn

def client_update(model, client_id, client_data, criterion, optimizer, local_steps=4, detailed_print=False):
    """
    Trains a given client's local model on its dataset for a fixed number of steps (`local_steps`).

    Args:
        model (nn.Module): The local model to be updated.
        client_id (int): Identifier for the client (used for logging/debugging purposes).
        client_data (DataLoader): The data loader for the client's dataset.
        criterion (Loss): The loss function used for training (e.g., CrossEntropyLoss).
        optimizer (Optimizer): The optimizer used for updating model parameters (e.g., SGD).
        local_steps (int): Number of local epochs to train on the client's dataset.
        detailed_print (bool): If True, logs the final loss after training.

    Returns:
        dict: The state dictionary of the updated model.
    """


    cudnn.benchmark  # Calling this optimizes runtime

    model.train()  # Set the model to training mode
    for epoch in range(local_steps):
        for data, targets in client_data:
            # Move data and targets to the specified device (e.g., GPU or CPU)
            data, targets = data.to(DEVICE), targets.to(DEVICE)


            start_time = time.time()  # for testing-----------------------------

            # Reset the gradients before backpropagation
            optimizer.zero_grad()
            state = model.init_hidden(data.size(0))
            state = (state[0].to(DEVICE), state[1].to(DEVICE)) # Move hidden state to device

            # Forward pass: compute model predictions
            outputs, _ = model(data, state) # Unpack the output to get the predictions

            # Compute the loss
            # Reshape the output to (batch_size * seq_len, n_vocab)
            outputs = outputs.view(-1, outputs.size(-1))
            # Reshape the target to (batch_size * seq_len)
            targets = targets.view(-1)

            loss = criterion(outputs, targets)

            # Backward pass: compute gradients and update weights
            loss.backward()
            optimizer.step()

            # for testing ------------------------------------------------------
            end_time = time.time()  # Record the end time
            elapsed_time = end_time - start_time  # Calculate the elapsed time
            print(f'Single step time taken: {elapsed_time:.2f} seconds')

    # Optionally, print the loss for the last epoch of training
    if detailed_print:
        print(f'Client {client_id} --> Final Loss (Epoch {epoch + 1}): {loss.item()}')


    # Return the updated model's state dictionary (weights)
    return model.state_dict()

### Central Server Aggregation with FedAvg

In [69]:
def fedavg_aggregate(global_model, client_states, client_sizes):
    """
    Aggregates model updates from selected clients using the Federated Averaging (FedAvg) algorithm.
    The updates are weighted by the size of each client's dataset.

    Args:
        global_model (nn.Module): The global model whose structure is used for aggregation.
        client_states (list[dict]): A list of state dictionaries (model weights) from participating clients.
        client_sizes (list[int]): A list of dataset sizes for the participating clients.

    Returns:
        dict: The aggregated state dictionary with updated model parameters.
    """
    # Copy the global model's state dictionary for aggregation
    new_state = deepcopy(global_model.state_dict())

    # Calculate the total number of samples across all participating clients
    total_samples = sum(client_sizes)

    # Initialize all parameters in the new state to zero
    for key in new_state:
        new_state[key] = torch.zeros_like(new_state[key])

    # Perform a weighted average of client updates
    for state, size in zip(client_states, client_sizes):
        for key in new_state:
            # Add the weighted contribution of each client's parameters
            new_state[key] += (state[key] * size / total_samples)

    # Return the aggregated state dictionary with updated weights
    return new_state

In [70]:
def evaluate(model, dataloader):
    with torch.no_grad():
        model.train(False) # Set Network to evaluation mode
        running_corrects = 0
        losses = []
        criterion = nn.NLLLoss()
        for data, targets in dataloader:
            data = data.to(DEVICE)        # Move the data to the GPU
            targets = targets.to(DEVICE)  # Move the targets to the GPU
            # Forward Pass
            state = model.init_hidden(data.size(0)) # Initialize hidden state
            state = (state[0].to(DEVICE), state[1].to(DEVICE)) # Move hidden state to device
            outputs, _ = model(data, state)  # Pass hidden state to the model
            # Reshape the output to (batch_size * seq_len, n_vocab)
            outputs = outputs.view(-1, outputs.size(-1))
            # Reshape the target to (batch_size * seq_len)
            targets = targets.view(-1)
            loss = criterion(outputs, targets)
            losses.append(loss.item())
            # Get predictions
            _, preds = torch.max(outputs, 1) # Changed outputs.data to outputs
            # Update Corrects
            running_corrects += torch.sum(preds == targets).item() # Changed targets.data to targets and .data.item() to .item()
            # Calculate Accuracy
            accuracy = running_corrects / float(len(dataloader.dataset))

    return accuracy, mean(losses) # Changed accuracy calculation to be outside the loop

### Federated Learning Training Loop

In [71]:
# Federated Learning Training Loop
def federated_training(global_model, criterion, trainloader, validloader, num_clients, rounds, C=0.1, local_steps=4,detailed_print=False):
    val_accuracies = []
    val_losses = []
    train_accuracies = []
    train_losses = []
    best_model_state = None  # The model with the best accuracy
    client_selection_count = [0] * num_clients #Count how many times a client has been selected
    best_val_acc = 0.0

    shards = sharding(trainloader.dataset, num_clients) #each shard represent the training data for one client
    client_sizes = [len(shard) for shard in shards]

    global_model.to(DEVICE) #as alwayse, we move the global model to the specified device (CPU or GPU)

    # ********************* HOW IT WORKS ***************************************
    # The training runs for rounds iterations (GLOBAL_ROUNDS=2000)
    # Each round simulates one communication step in federated learning, including:
    # 1) client selection
    # 2) local training (of each client)
    # 3) central aggregation
    for round_num in range(rounds):
        if round_num % LOG_FREQUENCY == 0 and detailed_print:
          print(f"------------------------------------- Round {round_num} ------------------------------------------------" )
        # 1) client selection: In each round, a fraction C (e.g., 10%) of clients is randomly selected to participate.
        #     This reduces computation costs and mimics real-world scenarios where not all devices are active.
        selected_clients = random.sample(range(num_clients), int(C * num_clients))
        client_states = []
        for client_id in selected_clients:
            client_selection_count[client_id] += 1

        # 2) local training: for each client updates the model using the client's data for local_steps epochs
        for client_id in selected_clients:
            local_model = deepcopy(global_model) #it creates a local copy of the global model
            optimizer = optim.SGD(local_model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY) #same of the centralized version
            client_loader = DataLoader(shards[client_id], batch_size=BATCH_SIZE, shuffle=True)

            local_state = client_update(local_model, client_id, client_loader, criterion, optimizer, local_steps, round_num % LOG_FREQUENCY == 0 and detailed_print)
            client_states.append(local_state)

        # 3) central aggregation: aggregates participating client updates using fedavg_aggregate
        #    and replaces the current parameters of global_model with the returned ones.
        global_model.load_state_dict(fedavg_aggregate(global_model, client_states, [client_sizes[i] for i in selected_clients]))

        # Validation at the server (optional, add metrics here)
         #if round_num % LOG_FREQUENCY:
          #val_loss, val_accuracy = evaluate_model(global_model, valid_dataset, criterion)
          #print(f"Round {round_num}: Validation Loss = {val_loss:.4f}, Accuracy = {val_accuracy:.2f}%")
        val_accuracy, val_loss = evaluate(global_model, validloader)
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            best_model_state = deepcopy(global_model.state_dict())
        if round_num % LOG_FREQUENCY == 0 and detailed_print:
            print(f"------------------------------ Round {round_num} terminated: model updated -----------------------------\n\n" )
            print(f"accuracy: {val_accuracy}, best accuracy: {best_val_acc}")
        #train_accuracy, train_loss = evaluate(global_model, trainloader)
        #print("evaluation on training dataset")
        val_accuracies.append(val_accuracy)
        val_losses.append(val_loss)
        #train_accuracies.append(train_accuracy)
        #train_losses.append(train_loss)

    global_model.load_state_dict(best_model_state)

    return val_accuracies,val_losses,global_model,client_selection_count

### Initialize Model & Loss

In [72]:
global_model = CharLSTM(input_size=N_VOCAB, embedding_size=8, hidden_size=256, num_layers=2, output_size=N_VOCAB)

criterion = nn.NLLLoss()# our loss function for classification tasks on CIFAR-100


### Run the training

In [ ]:
# Run Federated Learning
al_accuracies,val_losses,train_accuracies,train_losses,global_model,client_selection_count = federated_training(global_model, criterion, train_loader, val_loader, num_clients=NUM_CLIENTS, rounds=GLOBAL_ROUNDS, C=FRACTION_CLIENTS, local_steps=LOCAL_EPOCHS, detailed_print=True)


------------------------------------- Round 0 ------------------------------------------------
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Client 48 --> Final Loss (Epoch 4): 4.852753162384033
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Client 54 --> Final Loss (Epoch 4): 4.856601238250732
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:1123: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:1410.)
  result = _VF.lstm(


Single step time taken: 0.01 seconds
Client 88 --> Final Loss (Epoch 4): 4.852254867553711
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Client 27 --> Final Loss (Epoch 4): 4.857930660247803
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Client 70 --> Final Loss (Epoch 4): 4.857385158538818
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Client 44 --> Final Loss (Epoch 4): 4.854654788970947
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Client 79 --> Final Loss (Epoch 4): 4.857777118682861
Single step time taken: 0.01 seconds
Single step time taken: 0.01 seconds
Single step time taken: 0.0

# Validation

In [ ]:
def evaluate(model, dataloader, criterion):
    with torch.no_grad():
        model.train(False) # Set Network to evaluation mode
        running_corrects = 0
        losses = []
        for data, targets in dataloader:
            data = data.to(DEVICE)        # Move the data to the GPU
            targets = targets.to(DEVICE)  # Move the targets to the GPU
            # Forward Pass
            outputs = model(data)
            loss = criterion(outputs, targets)
            losses.append(loss.item())
            # Get predictions
            _, preds = torch.max(outputs.data, 1)
            # Update Corrects
            running_corrects += torch.sum(preds == targets.data).data.item()
            # Calculate Accuracy
            accuracy = running_corrects / float(len(dataloader.dataset))

    return accuracy, mean(losses)

### Run the test



In [ ]:
accuracy = evaluate(refined_model, testloader, criterion)[0]
print('\nTest Accuracy: {}'.format(accuracy))